In [7]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
import torch
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


Load model

In [6]:
import torch
config = EcgConfig()    
model_quantized = torch.load("..\\model_save\\model_quantized_98.pth")
model_quantized.eval()
model_quantized.to('cpu')

C:\Users\Xaio\anaconda3\envs\pytorch-ecg\lib\site-packages\torch\_utils.py:382: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)


量化模型

In [8]:
# import torch.quantization
# 
# # 使用 Eager Mode Quantization
# # 將 torch.nn.Linear 的參數映射到 -127~127 之間

# quantized_model = torch.quantization.quantize_dynamic(
#     model, {torch.nn.Linear}, dtype=torch.qint8
# )
# 
# torch.save(quantized_model, "..\\model_save\\model_quantized_98.pth")

準確度測試

In [10]:
print(model_quantized)

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)


In [12]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
for signal, label in loader[Mode.train]:
    signal.to('cpu')
    label.to('cpu')
    signal = einops.rearrange(signal, "b c e -> b e c")
    # print(signal)
    p = model_quantized(signal)
    print(p)
    print(label)
    print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



tensor([[  7.3524,   0.4145,  -3.0895,  -4.2983,  -9.4119,  -1.2548],
        [  1.0958,  -0.1882,  -1.5530,  -4.7840,  11.1596,  -2.5099],
        [  7.6049,   5.1967,  -7.6189,  -1.4542, -12.6208,  -0.5748],
        ...,
        [  7.7039,  -1.6093,  -8.5180,  -0.1873, -11.5546,   4.4579],
        [  7.1878,  -2.8949,  -1.0929,  -1.0383,  -8.1328,  -3.3142],
        [  4.4404,  -1.4253,  -3.3040,   1.3546,  -8.0465,   0.5579]],
       grad_fn=<WarnNotImplemented>)
tensor([0, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
        0, 2, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 5, 0, 2, 0, 4, 0, 0, 0, 0, 5, 5,
        0, 0, 2, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0,
        0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 3, 0, 0, 0, 2,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 5, 0,
        0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 0, 0, 1, 0, 0, 4, 0, 5, 1,
        2, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 0,

## Visualize the model

Neutron 可視化

In [22]:
inputs = torch.randn(1, 187, 1)
traced_script_module = torch.jit.trace(model_quantized, inputs)
traced_script_module.save("traced_resnet_model.pth")

import netron
modelData = 'traced_resnet_model.pth'
netron.start(modelData)

Serving 'traced_resnet_model.pth' at http://localhost:8080


('localhost', 8080)

Grapviz 可視化

In [ ]:
# from torchviz import make_dot
# vis_graph = make_dot(model_quantized(inputs), params=dict(model_quantized.named_parameters()), show_attrs=True, show_saved=True)
# vis_graph.view()  # 会在当前目录下保存一个“Digraph.gv.pdf”文件，并在默认浏览器中打开

## Parameter Extraction

### 提取全部的參數(忽略量化的資訊)

In [ ]:
model_state_dict = model_quantized.state_dict()

ignore_keys = [
    'scale',
    'zero_point',
    '_packed_params.dtype'
]

with open('model_layers_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if not any(ignore_key in layer_name for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n") 
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")


### 提取六層 Encoder 參數

In [25]:
model_state_dict = model_quantized.state_dict()

ignore_keys = [
    '.scale',
    '.zero_point',
    '._packed_params.dtype'
]

# Extract encoder.0 to encoder.5
for i in range(6):
    with open(f'encoder_{i}_params.txt', 'w') as f:
        for layer_name, param_tensor in model_state_dict.items():
            if layer_name.startswith(f'encoder.{i}') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
                f.write(f"Layer: {layer_name}\n") 
                f.write(f"Parameters: {str(param_tensor)}\n")
                f.write("\n")

# Extract classifier
with open('classifier_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith('classifier') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n")
            f.write(f"Parameters: {str(param_tensor)}\n") 
            f.write("\n")

# Extract embedding.cls_token
with open('cls_token_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name == 'embedding.cls_token':
            f.write(f"Layer: {layer_name}\n")
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")

# Extract embedding
with open('embedding_params.txt', 'w') as f: 
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith('embedding') and layer_name != 'embedding.cls_token' and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n")
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")

# Extract positional_encoding
with open('pos_encoding_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith('positional_encoding') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n") 
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")
